# 📈 Google Stock Trading System - Advanced ML Prediction

## 🎯 What This Notebook Does:
- Analyzes **10 years of Google (GOOGL) stock data**
- Trains **8 powerful ML models** for accurate predictions  
- Gives you a **clear BUY/SELL/HOLD recommendation**
- Shows **confidence levels** and model agreement
- Provides **exact entry/exit prices** for paper trading

## 🤖 Models Used:
1. Linear Regression (Baseline)
2. Random Forest (200 trees)
3. XGBoost (Gradient Boosting)
4. Gradient Boosting
5. LSTM Neural Network (Deep Learning)
6. Support Vector Regression (SVR)
7. AdaBoost
8. Extra Trees

**📅 Last Updated:** 2025-09-30  
**💰 Stock:** GOOGL (Alphabet Inc.)  
**📊 Data:** 10 years of daily prices

## Step 1: Import Libraries & Setup

In [4]:
# Import all required libraries
import warnings
warnings.filterwarnings('ignore')

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# ML models
from sklearn.ensemble import (
    RandomForestRegressor, 
    GradientBoostingRegressor, 
    AdaBoostRegressor, 
    ExtraTreesRegressor
)
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# XGBoost
import xgboost as xgb

# Data collection
import yfinance as yf

# Technical indicators
import pandas_ta as ta

print("✅ All libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"🤖 TensorFlow: {tf.__version__}")
print(f"⚡ XGBoost: {xgb.__version__}")


✅ All libraries imported successfully!
📅 Analysis Date: 2025-09-30 16:49:12
🐍 Python: 3.12.1
🤖 TensorFlow: 2.20.0
⚡ XGBoost: 3.0.5


## Step 2: Collect Google Stock Data (10 Years)

In [5]:
# Configuration
TICKER = 'GOOGL'
PERIOD = '10y'  # 10 years for maximum accuracy

print(f"🎯 Target Stock: {TICKER}")
print(f"📊 Data Period: {PERIOD}")
print(f"\n⏳ Downloading data from Yahoo Finance...\n")

# Fetch data
df = yf.download(TICKER, period=PERIOD, interval='1d', progress=False)

# Check if data was downloaded successfully
if df.empty:
    print("❌ Error: No data was downloaded!")
    print("Possible reasons:")
    print("  - No internet connection")
    print("  - Invalid ticker symbol")
    print("  - Yahoo Finance API issues")
    print("\nTrying alternative approach with start date...")
    
    from datetime import datetime
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)
    
    df = yf.download(TICKER, start=start_date, end=end_date, interval='1d', progress=False)
    
    if df.empty:
        raise ValueError(f"Unable to download data for {TICKER}. Please check your internet connection and ticker symbol.")

print(f"✅ Data downloaded successfully!")
print(f"📈 Total rows: {len(df):,}")
print(f"📅 Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
print(f"💰 Current Price: ${df['Close'].iloc[-1]:.2f}")
print(f"\n📊 Latest Data:")
df.tail()



1 Failed download:
['GOOGL']: ImpersonateError('Impersonating chrome136 is not supported')
['GOOGL']: ImpersonateError('Impersonating chrome136 is not supported')

1 Failed download:
['GOOGL']: ImpersonateError('Impersonating chrome136 is not supported')

1 Failed download:
['GOOGL']: ImpersonateError('Impersonating chrome136 is not supported')


🎯 Target Stock: GOOGL
📊 Data Period: 10y

⏳ Downloading data from Yahoo Finance...

❌ Error: No data was downloaded!
Possible reasons:
  - No internet connection
  - Invalid ticker symbol
  - Yahoo Finance API issues

Trying alternative approach with start date...


ValueError: Unable to download data for GOOGL. Please check your internet connection and ticker symbol.

## Step 3: Exploratory Data Analysis

In [ ]:
# Basic statistics
print("📊 Google Stock Statistics (10 Years):\n")
print(df[['Open', 'High', 'Low', 'Close', 'Volume']].describe().round(2))

# Calculate returns
df['Daily_Return'] = df['Close'].pct_change()
volatility = df['Daily_Return'].std() * np.sqrt(252) * 100

print(f"\n📊 Key Metrics:")
print(f"   Annualized Volatility: {volatility:.2f}%")
print(f"   Max Daily Gain: {df['Daily_Return'].max()*100:.2f}%")
print(f"   Max Daily Loss: {df['Daily_Return'].min()*100:.2f}%")
print(f"   Average Daily Return: {df['Daily_Return'].mean()*100:.4f}%")

In [ ]:
# Visualize price history
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Price chart
ax1 = axes[0]
ax1.plot(df.index, df['Close'], linewidth=2, color='#2E86DE')
ax1.set_title(f'{TICKER} - 10 Year Price History', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.fill_between(df.index, df['Close'], alpha=0.3, color='#2E86DE')

# Volume chart
ax2 = axes[1]
ax2.bar(df.index, df['Volume'], color='gray', alpha=0.5)
ax2.set_title('Trading Volume', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Volume', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Price history visualized!")

## Step 4: Advanced Feature Engineering (31 Technical Indicators)

In [ ]:
print("🔧 Creating technical indicators...\n")

# Create a copy for features
data = df.copy()

# 1. Moving Averages
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['EMA_12'] = data['Close'].ewm(span=12).mean()
data['EMA_26'] = data['Close'].ewm(span=26).mean()

# 2. Momentum Indicators
data['RSI'] = ta.rsi(data['Close'], length=14)
data['MACD'] = ta.macd(data['Close'])['MACD_12_26_9']
data['MACD_signal'] = ta.macd(data['Close'])['MACDs_12_26_9']

# 3. Volatility Indicators
bbands = ta.bbands(data['Close'], length=20)
data['BB_upper'] = bbands['BBU_20_2.0']
data['BB_middle'] = bbands['BBM_20_2.0']
data['BB_lower'] = bbands['BBL_20_2.0']
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)

# 4. Volume Indicators
data['OBV'] = ta.obv(data['Close'], data['Volume'])

# 5. Trend Indicators
data['ADX'] = ta.adx(data['High'], data['Low'], data['Close'])['ADX_14']

# 6. Price Changes
data['Price_Change'] = data['Close'].pct_change()
data['Price_Change_5'] = data['Close'].pct_change(periods=5)
data['Price_Change_10'] = data['Close'].pct_change(periods=10)

# 7. High-Low Spread
data['HL_Spread'] = (data['High'] - data['Low']) / data['Close']

# 8. Volume Changes
data['Volume_Change'] = data['Volume'].pct_change()

# 9. Stochastic Oscillator
stoch = ta.stoch(data['High'], data['Low'], data['Close'])
data['Stoch_K'] = stoch['STOCHk_14_3_3']
data['Stoch_D'] = stoch['STOCHd_14_3_3']

# 10. Lag features
for i in [1, 2, 3, 5, 10]:
    data[f'Close_Lag_{i}'] = data['Close'].shift(i)

# Target: Next day's closing price
data['Target'] = data['Close'].shift(-1)

# Drop rows with NaN values
data = data.dropna()

print(f"✅ Feature engineering complete!")
print(f"📊 Total features: {len(data.columns) - 1} (excluding target)")
print(f"📈 Usable rows: {len(data):,}")
print(f"\n🔧 Features created: {list(data.columns[:20])}...")
print(f"\n📊 Recent data with features:")
data[['Close', 'SMA_20', 'RSI', 'MACD', 'Target']].tail()

## Step 5: Prepare Data for Training

In [ ]:
# Prepare features and target
feature_cols = [col for col in data.columns if col not in ['Target', 'Daily_Return']]
X = data[feature_cols].values
y = data['Target'].values

# Train-test split (80-20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"📊 Data Split:")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")
print(f"   Features: {X_train.shape[1]}")
print(f"   Split ratio: 80/20")
print(f"\n✅ Data preparation complete!")

## Step 6: Train All 8 ML Models

In [ ]:
# Helper function to evaluate models
def evaluate_model(y_true, y_pred, model_name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"✅ {model_name:20s} - R²: {r2:.4f} | RMSE: ${rmse:.2f} | MAE: ${mae:.2f}")
    
    return {'r2': r2, 'rmse': rmse, 'mae': mae, 'mape': mape}

# Store results
models = {}
predictions = {}
metrics = {}

print("🤖 Training 8 ML Models...\n")
print("="*80)

In [ ]:
# Model 1: Linear Regression
print("\n1️⃣ Training Linear Regression...")
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
models['Linear Regression'] = lr
predictions['Linear Regression'] = lr_pred
metrics['Linear Regression'] = evaluate_model(y_test, lr_pred, 'Linear Regression')

In [ ]:
# Model 2: Random Forest
print("\n2️⃣ Training Random Forest...")
rf = RandomForestRegressor(n_estimators=200, max_depth=25, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
models['Random Forest'] = rf
predictions['Random Forest'] = rf_pred
metrics['Random Forest'] = evaluate_model(y_test, rf_pred, 'Random Forest')

In [ ]:
# Model 3: XGBoost
print("\n3️⃣ Training XGBoost...")
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=8, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train, verbose=False)
xgb_pred = xgb_model.predict(X_test)
models['XGBoost'] = xgb_model
predictions['XGBoost'] = xgb_pred
metrics['XGBoost'] = evaluate_model(y_test, xgb_pred, 'XGBoost')

In [ ]:
# Model 4: Gradient Boosting
print("\n4️⃣ Training Gradient Boosting...")
gb = GradientBoostingRegressor(n_estimators=200, max_depth=8, learning_rate=0.05, random_state=42)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
models['Gradient Boosting'] = gb
predictions['Gradient Boosting'] = gb_pred
metrics['Gradient Boosting'] = evaluate_model(y_test, gb_pred, 'Gradient Boosting')

In [ ]:
# Model 5: LSTM Neural Network
print("\n5️⃣ Training LSTM Neural Network...")

# Scale data for LSTM
scaler_lstm = StandardScaler()
X_train_lstm = scaler_lstm.fit_transform(X_train)
X_test_lstm = scaler_lstm.transform(X_test)

# Reshape for LSTM [samples, timesteps, features]
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], 1, X_train_lstm.shape[1]))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], 1, X_test_lstm.shape[1]))

# Build LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(1, X_train.shape[1])),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train_lstm, y_train, epochs=30, batch_size=32, verbose=0)

lstm_pred = lstm_model.predict(X_test_lstm, verbose=0).flatten()
models['LSTM'] = lstm_model
predictions['LSTM'] = lstm_pred
metrics['LSTM'] = evaluate_model(y_test, lstm_pred, 'LSTM')

In [ ]:
# Model 6: Support Vector Regression (SVR)
print("\n6️⃣ Training Support Vector Regression...")

# Scale data for SVR
scaler_svr = StandardScaler()
X_train_svr = scaler_svr.fit_transform(X_train)
X_test_svr = scaler_svr.transform(X_test)

svr_model = SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1)
svr_model.fit(X_train_svr, y_train)
svr_pred = svr_model.predict(X_test_svr)
models['SVR'] = svr_model
predictions['SVR'] = svr_pred
metrics['SVR'] = evaluate_model(y_test, svr_pred, 'SVR')

In [ ]:
# Model 7: AdaBoost
print("\n7️⃣ Training AdaBoost...")
ada = AdaBoostRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)
models['AdaBoost'] = ada
predictions['AdaBoost'] = ada_pred
metrics['AdaBoost'] = evaluate_model(y_test, ada_pred, 'AdaBoost')

In [ ]:
# Model 8: Extra Trees
print("\n8️⃣ Training Extra Trees...")
et = ExtraTreesRegressor(n_estimators=200, max_depth=25, random_state=42, n_jobs=-1)
et.fit(X_train, y_train)
et_pred = et.predict(X_test)
models['Extra Trees'] = et
predictions['Extra Trees'] = et_pred
metrics['Extra Trees'] = evaluate_model(y_test, et_pred, 'Extra Trees')

print("\n" + "="*80)
print("✅ All models trained successfully!")

## Step 7: Compare Model Performance

In [ ]:
# Create comparison DataFrame
comparison_data = []
for model_name in metrics.keys():
    comparison_data.append({
        'Model': model_name,
        'R² Score': metrics[model_name]['r2'],
        'RMSE': metrics[model_name]['rmse'],
        'MAE': metrics[model_name]['mae'],
        'MAPE': metrics[model_name]['mape']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('R² Score', ascending=False)

print("\n" + "="*80)
print("📊 MODEL PERFORMANCE COMPARISON (Ranked by R² Score)")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

best_model = comparison_df.iloc[0]['Model']
best_r2 = comparison_df.iloc[0]['R² Score']
print(f"\n🏆 Best Model: {best_model} (R² = {best_r2:.4f})")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# R² Score
ax1 = axes[0, 0]
colors = ['#2E86DE' if x == comparison_df['R² Score'].max() else '#95A5A6' for x in comparison_df['R² Score']]
ax1.barh(comparison_df['Model'], comparison_df['R² Score'], color=colors)
ax1.set_xlabel('R² Score')
ax1.set_title('R² Score Comparison (Higher = Better)', fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# RMSE
ax2 = axes[0, 1]
colors = ['#27AE60' if x == comparison_df['RMSE'].min() else '#95A5A6' for x in comparison_df['RMSE']]
ax2.barh(comparison_df['Model'], comparison_df['RMSE'], color=colors)
ax2.set_xlabel('RMSE ($)')
ax2.set_title('RMSE Comparison (Lower = Better)', fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# MAE
ax3 = axes[1, 0]
colors = ['#E67E22' if x == comparison_df['MAE'].min() else '#95A5A6' for x in comparison_df['MAE']]
ax3.barh(comparison_df['Model'], comparison_df['MAE'], color=colors)
ax3.set_xlabel('MAE ($)')
ax3.set_title('MAE Comparison (Lower = Better)', fontweight='bold')
ax3.grid(axis='x', alpha=0.3)

# MAPE
ax4 = axes[1, 1]
colors = ['#9B59B6' if x == comparison_df['MAPE'].min() else '#95A5A6' for x in comparison_df['MAPE']]
ax4.barh(comparison_df['Model'], comparison_df['MAPE'], color=colors)
ax4.set_xlabel('MAPE (%)')
ax4.set_title('MAPE Comparison (Lower = Better)', fontweight='bold')
ax4.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Step 8: Make Next-Day Predictions

In [ ]:
# Get latest data point for prediction
latest_features = X[-1:]
current_price = df['Close'].iloc[-1]

# Get predictions from all models
next_day_predictions = {}

next_day_predictions['Linear Regression'] = models['Linear Regression'].predict(latest_features)[0]
next_day_predictions['Random Forest'] = models['Random Forest'].predict(latest_features)[0]
next_day_predictions['XGBoost'] = models['XGBoost'].predict(latest_features)[0]
next_day_predictions['Gradient Boosting'] = models['Gradient Boosting'].predict(latest_features)[0]

# LSTM prediction
latest_lstm = scaler_lstm.transform(latest_features)
latest_lstm = latest_lstm.reshape((latest_lstm.shape[0], 1, latest_lstm.shape[1]))
next_day_predictions['LSTM'] = models['LSTM'].predict(latest_lstm, verbose=0)[0][0]

# SVR prediction
latest_svr = scaler_svr.transform(latest_features)
next_day_predictions['SVR'] = models['SVR'].predict(latest_svr)[0]

next_day_predictions['AdaBoost'] = models['AdaBoost'].predict(latest_features)[0]
next_day_predictions['Extra Trees'] = models['Extra Trees'].predict(latest_features)[0]

# Calculate weighted ensemble (weighted by R² score)
weights = comparison_df['R² Score'].values
weights = weights / weights.sum()

model_order = comparison_df['Model'].tolist()
pred_values = [next_day_predictions[m] for m in model_order]
ensemble_pred = np.average(pred_values, weights=weights)

# Display predictions
print("\n" + "="*80)
print("🔮 NEXT DAY PRICE PREDICTIONS (All Models)")
print("="*80)
print(f"\n💰 Current Price: ${current_price:.2f}\n")

for model_name in model_order:
    pred = next_day_predictions[model_name]
    change = pred - current_price
    change_pct = (change / current_price) * 100
    direction = "📈" if change > 0 else "📉"
    print(f"{direction} {model_name:20s}: ${pred:7.2f}  ({change:+7.2f} | {change_pct:+6.2f}%)")

ensemble_change = ensemble_pred - current_price
ensemble_change_pct = (ensemble_change / current_price) * 100
ensemble_direction = "📈" if ensemble_change > 0 else "📉"

print("\n" + "-"*80)
print(f"{ensemble_direction} {'Ensemble (Weighted)':20s}: ${ensemble_pred:7.2f}  ({ensemble_change:+7.2f} | {ensemble_change_pct:+6.2f}%)")
print("="*80)

## Step 9: 🎯 **TRADING RECOMMENDATION - BUY/SELL/HOLD**

In [ ]:
# Calculate consensus
bullish_count = sum(1 for pred in pred_values if pred > current_price)
bearish_count = sum(1 for pred in pred_values if pred < current_price)
total_models = len(pred_values)
consensus_pct = (max(bullish_count, bearish_count) / total_models) * 100

# Determine recommendation
if ensemble_change_pct > 2.0 and consensus_pct >= 62.5:
    recommendation = "🟢 STRONG BUY"
    action = "BUY"
    confidence = "HIGH"
elif ensemble_change_pct > 0.5 and consensus_pct >= 50:
    recommendation = "🟢 BUY"
    action = "BUY"
    confidence = "MEDIUM"
elif ensemble_change_pct < -2.0 and consensus_pct >= 62.5:
    recommendation = "🔴 STRONG SELL"
    action = "SELL"
    confidence = "HIGH"
elif ensemble_change_pct < -0.5 and consensus_pct >= 50:
    recommendation = "🔴 SELL"
    action = "SELL"
    confidence = "MEDIUM"
else:
    recommendation = "🟡 HOLD"
    action = "HOLD"
    confidence = "LOW"

# Print recommendation
print("\n" + "="*80)
print("🎯 TRADING RECOMMENDATION FOR GOOGLE (GOOGL)")
print("="*80)
print(f"\n📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💰 Current Price: ${current_price:.2f}")
print(f"🔮 Predicted Price: ${ensemble_pred:.2f}")
print(f"📊 Expected Change: {ensemble_change:+.2f} ({ensemble_change_pct:+.2f}%)")

print(f"\n📊 Model Consensus:")
print(f"   Bullish Models: {bullish_count}/{total_models} ({bullish_count/total_models*100:.1f}%)")
print(f"   Bearish Models: {bearish_count}/{total_models} ({bearish_count/total_models*100:.1f}%)")
print(f"   Consensus Strength: {consensus_pct:.1f}%")

print(f"\n" + "="*80)
print(f"📢 RECOMMENDATION: {recommendation}")
print(f"🎯 ACTION: {action}")
print(f"💪 CONFIDENCE: {confidence}")
print("="*80)

# Trading instructions
print(f"\n📝 Analysis Summary:")
print(f"   • Based on 10 years of historical data ({len(df):,} days)")
print(f"   • 8 advanced ML models with {X.shape[1]} features")
print(f"   • Best model: {best_model} (R²={best_r2:.4f})")
print(f"   • Weighted ensemble prediction")

if action == "BUY":
    print(f"\n✅ For Paper Trading (TradingView):")
    print(f"   📍 Entry Price: ${current_price:.2f}")
    print(f"   🎯 Target Price: ${ensemble_pred:.2f} (+{ensemble_change_pct:.2f}%)")
    print(f"   🛡️ Stop Loss: ${current_price * 0.98:.2f} (-2%)")
    print(f"   💼 Position Size: 5-10% of portfolio")
    print(f"   ⏰ Time Horizon: 1-5 days")
elif action == "SELL":
    print(f"\n⚠️ For Paper Trading (TradingView):")
    print(f"   📍 If holding: Exit at ${current_price:.2f}")
    print(f"   📉 Expected drop to: ${ensemble_pred:.2f} ({ensemble_change_pct:.2f}%)")
    print(f"   🔄 Alternative: Wait for reversal signal")
    print(f"   🚫 Avoid new long positions")
else:
    print(f"\n⏸️ For Paper Trading (TradingView):")
    print(f"   📍 Current position: Keep as is")
    print(f"   ⏳ Wait for clearer signal (>50% consensus)")
    print(f"   👀 Monitor for breakout or breakdown")
    print(f"   📊 Watch for volume confirmation")

print(f"\n⚠️ DISCLAIMER:")
print(f"   This is for EDUCATIONAL and PAPER TRADING purposes only.")
print(f"   Past performance does NOT guarantee future results.")
print(f"   Always manage risk and never invest more than you can afford to lose.")
print("\n" + "="*80)

## Step 10: Visualize All Predictions

In [ ]:
# Create prediction visualization
fig, ax = plt.subplots(figsize=(14, 8))

model_names = model_order + ['Ensemble']
all_predictions = pred_values + [ensemble_pred]
x_pos = np.arange(len(model_names))

# Current price line
ax.axhline(y=current_price, color='gray', linestyle='--', linewidth=2, 
           label=f'Current Price: ${current_price:.2f}', zorder=1)

# Color bars based on prediction
colors = ['green' if p > current_price else 'red' for p in all_predictions]
colors[-1] = 'darkgreen' if ensemble_pred > current_price else 'darkred'  # Ensemble

# Plot predictions
bars = ax.bar(x_pos, all_predictions, color=colors, alpha=0.7, zorder=2)

# Highlight ensemble
bars[-1].set_alpha(1.0)
bars[-1].set_edgecolor('black')
bars[-1].set_linewidth(2)

# Labels
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.set_ylabel('Predicted Price ($)', fontsize=12)
ax.set_title(f'Next Day Price Predictions for {TICKER}\nCurrent: ${current_price:.2f} | Ensemble: ${ensemble_pred:.2f} ({ensemble_change_pct:+.2f}%) | {recommendation}', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3, zorder=0)

# Add value labels
for i, (pred, name) in enumerate(zip(all_predictions, model_names)):
    va = 'bottom' if pred > current_price else 'top'
    y_offset = 1 if pred > current_price else -1
    ax.text(i, pred + y_offset, f'${pred:.2f}', ha='center', va=va, 
            fontsize=9, fontweight='bold' if name == 'Ensemble' else 'normal')

plt.tight_layout()
plt.show()

print("\n✅ Prediction visualization complete!")

## 🎓 Key Takeaways

### What You Just Did:
1. ✅ Analyzed 10 years of Google stock data
2. ✅ Created 31 technical indicators for better predictions
3. ✅ Trained 8 different ML models
4. ✅ Got a clear BUY/SELL/HOLD recommendation
5. ✅ Received exact entry/exit prices for paper trading

### How to Use This:
- **Run this notebook daily** before market open
- **Track predictions** in a journal to see accuracy
- **Paper trade first** on TradingView for 3-6 months
- **Adjust thresholds** if needed based on your risk tolerance
- **Combine with** fundamental analysis for best results

### Important Reminders:
- 📊 Models are trained on historical data
- ⚠️ Past performance ≠ Future results
- 🛡️ Always use stop-loss orders
- 💰 Never risk more than 1-2% per trade
- 📚 Keep learning and improving

### Next Steps:
1. **Save this recommendation** and compare with actual results tomorrow
2. **Practice on TradingView** paper trading
3. **Try different stocks** by changing TICKER variable
4. **Experiment with parameters** to improve accuracy
5. **Build a trading journal** to track your progress

---

**Good luck with your paper trading! 🚀📈**